based on https://www.kaggle.com/isaienkov/keras-nn-with-embeddings-for-cat-features-1-15

(this is a keras tensorflow so no need to change /.keras/keras.json)

# Dense NN 

In [1]:
!pip install pyarrow

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss

from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

./weather_test.csv
./requirement.txt
./level1--submission_multimeter004_nobuild--lightgbm.ipynb
./ashrae-energy-prediction.zip
./generate_leak_data.ipynb
./level1--submission_withoutleak001--lightgbm.ipynb
./train_simple_cleanup.feather
./train.csv
./train_cleanup_001.feather
./README.md
./model_summary.pdf
./cpumemuse_densenn.txt
./weather_train.csv
./level1--submission_whatsyourcv3_0052_trncl--lightgbm.ipynb
./level1--submission_multimeter003--lightgbm.ipynb
./cpumemuse.sh
./generate_datasets.ipynb
./level1--submission_nn001--DenseNN.ipynb
./test.csv
./model_0.hdf5
./level1--submission_nn007lofo--CNN.ipynb
./level2--ensembling_model.ipynb
./building_metadata.csv
./memuse_log.sh
./sample_submission.csv
./.ipynb_checkpoints/level1--submission_nn001--DenseNN-checkpoint.ipynb
./Catboost on GPU/requirement.txt
./Catboost on GPU/level1--catboost002--Catboost.ipynb
./Keras_NN_weights/cnn_model_0.hdf5
./Keras_NN_weights/cnn_model_1.hdf5
./Keras_NN_weights/model_1.hdf5
./Keras_NN_weights/mode

Using TensorFlow backend.


In [3]:
def average_imputation(df, column_name):
    imputation = df.groupby(['timestamp'])[column_name].mean()
    
    df.loc[df[column_name].isnull(), column_name] = df[df[column_name].isnull()][[column_name]].apply(lambda x: imputation[df['timestamp'][x.index]].values)
    del imputation
    return df


In [4]:
building_df = pd.read_csv("./building_metadata.csv")
weather_train = pd.read_csv("./weather_train.csv")
train = pd.read_feather("./train_cleanup_001.feather")

train = train.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
train = train.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"])
del weather_train

In [5]:

train.loc[(train['meter']==0) & (train['site_id']==0) & (train['timestamp']<'2016-05-21 00:00:00'), 'drop'] = True
train = train[train['drop']!=True]


In [6]:
train["timestamp"] = pd.to_datetime(train["timestamp"])
train["hour"] = train["timestamp"].dt.hour
train["weekday"] = train["timestamp"].dt.weekday

train = average_imputation(train, 'wind_speed')

beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

for item in beaufort:
    train.loc[(train['wind_speed']>=item[1]) & (train['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

del train["timestamp"]

In [7]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            meancol = df[col].mean()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(meancol,inplace=True)  
                
                print('change for', meancol)
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train["primary_use"] = le.fit_transform(train["primary_use"])

categoricals = ["site_id", "building_id", "primary_use", "hour", "weekday",  "meter"]

drop_cols = ["sea_level_pressure", "wind_speed", "wind_direction"]

numericals = ["square_feet", "year_built", "air_temperature", "cloud_coverage",
              "dew_temperature", "precip_depth_1_hr", "floor_count", 'beaufort_scale']

feat_cols = categoricals + numericals

In [9]:
target = np.log1p(train["meter_reading"])

del train["meter_reading"] 

train = train.drop(drop_cols, axis = 1)

In [10]:
train, NAlist = reduce_mem_usage(train)

Memory usage of properties dataframe is : 3033.1971435546875  MB
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  primary_use
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  int64
min for this col:  283
max for this col:  875000
dtype after:  uint32
******************************
******************************
Column:  year_built
dtype before:  float64
min for this col:  1900.0
m

In [11]:
def model(dense_dim_1=64, dense_dim_2=32, dense_dim_3=32, dense_dim_4=16, 
dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=0.001):

    #Inputs
    site_id = Input(shape=[1], name="site_id")
    building_id = Input(shape=[1], name="building_id")
    meter = Input(shape=[1], name="meter")
    primary_use = Input(shape=[1], name="primary_use")
    square_feet = Input(shape=[1], name="square_feet")
    year_built = Input(shape=[1], name="year_built")
    air_temperature = Input(shape=[1], name="air_temperature")
    cloud_coverage = Input(shape=[1], name="cloud_coverage")
    dew_temperature = Input(shape=[1], name="dew_temperature")
    hour = Input(shape=[1], name="hour")
    precip = Input(shape=[1], name="precip_depth_1_hr")
    weekday = Input(shape=[1], name="weekday")
    beaufort_scale = Input(shape=[1], name="beaufort_scale")
   
    #Embeddings layers
    emb_site_id = Embedding(16, 2)(site_id)
    emb_building_id = Embedding(1449, 6)(building_id)
    emb_meter = Embedding(4, 2)(meter)
    emb_primary_use = Embedding(16, 2)(primary_use)
    emb_hour = Embedding(24, 3)(hour)
    emb_weekday = Embedding(7, 2)(weekday)

    concat_emb = concatenate([
           Flatten() (emb_site_id)
         , Flatten() (emb_building_id)
         , Flatten() (emb_meter)
         , Flatten() (emb_primary_use)
         , Flatten() (emb_hour)
         , Flatten() (emb_weekday)
    ])
    
    categ = Dropout(dropout1)(Dense(dense_dim_1,activation='relu') (concat_emb))
    categ = BatchNormalization()(categ)
    categ = Dropout(dropout2)(Dense(dense_dim_2,activation='relu') (categ))
    
    #main layer
    main_l = concatenate([
          categ
        , square_feet
        , year_built
        , air_temperature
        , cloud_coverage
        , dew_temperature
        , precip
        , beaufort_scale
    ])
    
    main_l = Dropout(dropout3)(Dense(dense_dim_3,activation='relu') (main_l))
    main_l = BatchNormalization()(main_l)
    
    main_2 = concatenate([
          main_l
        , Flatten() (emb_building_id)
        , Flatten() (emb_site_id)

    ])
    
    main_2 = Dropout(dropout4)(Dense(dense_dim_4,activation='relu') (main_2))
    
    #output
    output = Dense(1) (main_2)

    model = Model([ site_id,
                    building_id, 
                    meter, 
                    primary_use, 
                    square_feet, 
                    year_built, 
                   
                    air_temperature,
                    cloud_coverage,
                    dew_temperature, 
                    hour,
                    weekday, 
                    precip,
                    beaufort_scale], output)

    model.compile(optimizer = Adam(lr=lr),
                  loss= mse_loss,
                  metrics=[root_mean_squared_error])
    return model

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))

In [12]:
def get_keras_data(df, num_cols, cat_cols):
    cols = cat_cols + num_cols
    X = {col: np.array(df[col]) for col in cols}
    return X

def train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold, patience=3):
    early_stopping = EarlyStopping(patience=patience, verbose=1)
    model_checkpoint = ModelCheckpoint("model_" + str(fold) + ".hdf5",
                                       save_best_only=True, verbose=1, monitor='val_root_mean_squared_error', mode='min')

    hist = keras_model.fit(X_t, y_train, batch_size=batch_size, epochs=epochs,
                            validation_data=(X_v, y_valid), verbose=1,
                            callbacks=[early_stopping, model_checkpoint])

    keras_model = load_model("model_" + str(fold) + ".hdf5", custom_objects={'root_mean_squared_error': root_mean_squared_error})
    
    return keras_model

In [13]:
from sklearn.model_selection import KFold, StratifiedKFold

oof = np.zeros(len(train))
batch_size = 256
epochs = 15
models = []

folds = 2
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

for fold_n, (train_index, valid_index) in enumerate(kf.split(train, train['building_id'])):
    print('Fold:', fold_n)
    X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_t = get_keras_data(X_train, numericals, categoricals)
    X_v = get_keras_data(X_valid, numericals, categoricals)
    
    keras_model = model(dense_dim_1=64, dense_dim_2=32, dense_dim_3=32, dense_dim_4=16, 
                        dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=0.001)
    mod = train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold_n, patience=3)
    models.append(mod)
    print('*'* 50)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fold: 0




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Train on 9802535 samples, validate on 9802536 samples
Epoch 1/15





9802535/9802535 [==============================] - 560s 57us/step - loss: 1.3307 - root_mean_squared_error: 1.1347 - val_loss: 1.3429 - val_root_mean_squared_error: 1.1053

Epoch 00001: val_root_mean_squared_error improved from inf to 1.10534, saving model to model_0.hdf5
Epoch 2/15
9802535/9802535 [==============================] - 560s 57us/step - loss: 0.9660 - root_mean_squared_error: 0.9771 - val_loss: 1.1692 - val_root_mean_squared_error: 1.0305

Epoch 00002: val_root_mean_squared_error improved from 1.10534 to 1.03046, saving model to model_0.hdf5
Epoch 3/15
9802535/9802535 [==============================] - 563s 57us/step - loss: 0.9023 - root_mean_squared_error: 0.9440 - val_loss: 1.1432 - val_root_mean_squared_error: 1.0206

Epoch 00003: val_root_mean_squared_error improved from

9802536/9802536 [==============================] - 620s 63us/step - loss: 0.9018 - root_mean_squared_error: 0.9441 - val_loss: 1.1185 - val_root_mean_squared_error: 1.0117

Epoch 00003: val_root_mean_squared_error improved from 1.06265 to 1.01167, saving model to model_1.hdf5
Epoch 4/15
9802536/9802536 [==============================] - 617s 63us/step - loss: 0.8577 - root_mean_squared_error: 0.9206 - val_loss: 1.1584 - val_root_mean_squared_error: 1.0260

Epoch 00004: val_root_mean_squared_error did not improve from 1.01167
Epoch 5/15
9802536/9802536 [==============================] - 620s 63us/step - loss: 0.8338 - root_mean_squared_error: 0.9075 - val_loss: 1.1020 - val_root_mean_squared_error: 1.0025

Epoch 00005: val_root_mean_squared_error improved from 1.01167 to 1.00249, saving model to model_1.hdf5
Epoch 6/15
9802536/9802536 [==============================] - 618s 63us/step - loss: 0.8197 - root_mean_squared_error: 0.8997 - val_loss: 1.1094 - val_root_mean_squared_error: 1.003

In [15]:
from sklearn.model_selection import KFold, StratifiedKFold

oof = np.zeros(len(train))
batch_size = 256
epochs = 15
models = []

folds = 2
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

for fold_n, (train_index, valid_index) in enumerate(kf.split(train, train['building_id'])):
    print('Fold:', fold_n)
    X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_t = get_keras_data(X_train, numericals, categoricals)
    X_v = get_keras_data(X_valid, numericals, categoricals)
    
    keras_model = model(dense_dim_1=64, dense_dim_2=32, dense_dim_3=32, dense_dim_4=16, 
                        dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=0.001)
    mod = train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold_n, patience=3)
    models.append(mod)
    print('*'* 50)
    

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fold: 0
Train on 9802535 samples, validate on 9802536 samples
Epoch 1/15
9802535/9802535 [==============================] - 628s 64us/step - loss: 1.4041 - root_mean_squared_error: 1.1641 - val_loss: 1.2889 - val_root_mean_squared_error: 1.0835

Epoch 00001: val_root_mean_squared_error improved from inf to 1.08355, saving model to model_0.hdf5
Epoch 2/15
9802535/9802535 [==============================] - 621s 63us/step - loss: 0.9752 - root_mean_squared_error: 0.9816 - val_loss: 1.1493 - val_root_mean_squared_error: 1.0194

Epoch 00002: val_root_mean_squared_error improved from 1.08355 to 1.01940, saving model to model_0.hdf5
Epoch 3/15
9802535/9802535 [==============================] - 624s 64us/step - loss: 0.8981 - root_mean_squared_error: 0.9419 - val_loss: 1.1769 - val_root_mean_squared_error: 1.0392

Epoch 00003: val_root_mean_squared_error did not improve from 1.01940
Epoch 4/15
9802535/9802535 [==============================] - 623s 64us/step - loss: 0.8614 - root_mean_squared_

In [16]:
numericals

['square_feet',
 'year_built',
 'air_temperature',
 'cloud_coverage',
 'dew_temperature',
 'precip_depth_1_hr',
 'floor_count',
 'beaufort_scale']

In [17]:
import gc
del train, target, X_train, X_valid, y_train, y_valid, X_t, X_v, kf
gc.collect()

260

In [18]:
test = pd.read_csv("./test.csv")
test = test.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
del building_df
gc.collect()
test["primary_use"] = le.transform(test["primary_use"])

weather_test = pd.read_csv("./weather_test.csv")

test = test.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
del weather_test

In [19]:
test["timestamp"] = pd.to_datetime(test["timestamp"])
test["hour"] = test["timestamp"].dt.hour
test["weekday"] = test["timestamp"].dt.weekday

test = average_imputation(test, 'wind_speed')

for item in beaufort:
    test.loc[(test['wind_speed']>=item[1]) & (test['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

    
test = test[feat_cols]
test, NAlist = reduce_mem_usage(test)

Memory usage of properties dataframe is : 6051.91162109375  MB
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  primary_use
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  hour
dtype before:  int64
min for this col:  0
max for this col:  23
dtype after:  uint8
******************************
******************************
Column:  weekday
dtype before:  int64
min for this col:  0
max for this col:  6
dtype after:  uint8
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype 

In [31]:
!pip install tqdm==4.32.2

In [41]:
#from tqdm import tqdm
i=0
res = np.zeros((test.shape[0]),dtype=np.float32)
step_size = 50000
for j in range(int(np.ceil(test.shape[0]/step_size))):
    print('iteration {} - count {}'.format(j,i))
    for_prediction = get_keras_data(test.iloc[i:i+step_size], numericals, categoricals)
    res[i:min(i+step_size,test.shape[0])] = \
       np.expm1(sum([model.predict(for_prediction, batch_size=1024)[:,0] for model in models])/folds)
    i+=step_size

iteration 0 - count 0
iteration 1 - count 50000
iteration 2 - count 100000
iteration 3 - count 150000
iteration 4 - count 200000
iteration 5 - count 250000
iteration 6 - count 300000
iteration 7 - count 350000
iteration 8 - count 400000
iteration 9 - count 450000
iteration 10 - count 500000
iteration 11 - count 550000
iteration 12 - count 600000
iteration 13 - count 650000
iteration 14 - count 700000
iteration 15 - count 750000
iteration 16 - count 800000
iteration 17 - count 850000
iteration 18 - count 900000
iteration 19 - count 950000
iteration 20 - count 1000000
iteration 21 - count 1050000
iteration 22 - count 1100000
iteration 23 - count 1150000
iteration 24 - count 1200000
iteration 25 - count 1250000
iteration 26 - count 1300000
iteration 27 - count 1350000
iteration 28 - count 1400000
iteration 29 - count 1450000
iteration 30 - count 1500000
iteration 31 - count 1550000
iteration 32 - count 1600000
iteration 33 - count 1650000
iteration 34 - count 1700000
iteration 35 - count 

iteration 276 - count 13800000
iteration 277 - count 13850000
iteration 278 - count 13900000
iteration 279 - count 13950000
iteration 280 - count 14000000
iteration 281 - count 14050000
iteration 282 - count 14100000
iteration 283 - count 14150000
iteration 284 - count 14200000
iteration 285 - count 14250000
iteration 286 - count 14300000
iteration 287 - count 14350000
iteration 288 - count 14400000
iteration 289 - count 14450000
iteration 290 - count 14500000
iteration 291 - count 14550000
iteration 292 - count 14600000
iteration 293 - count 14650000
iteration 294 - count 14700000
iteration 295 - count 14750000
iteration 296 - count 14800000
iteration 297 - count 14850000
iteration 298 - count 14900000
iteration 299 - count 14950000
iteration 300 - count 15000000
iteration 301 - count 15050000
iteration 302 - count 15100000
iteration 303 - count 15150000
iteration 304 - count 15200000
iteration 305 - count 15250000
iteration 306 - count 15300000
iteration 307 - count 15350000
iteratio

iteration 541 - count 27050000
iteration 542 - count 27100000
iteration 543 - count 27150000
iteration 544 - count 27200000
iteration 545 - count 27250000
iteration 546 - count 27300000
iteration 547 - count 27350000
iteration 548 - count 27400000
iteration 549 - count 27450000
iteration 550 - count 27500000
iteration 551 - count 27550000
iteration 552 - count 27600000
iteration 553 - count 27650000
iteration 554 - count 27700000
iteration 555 - count 27750000
iteration 556 - count 27800000
iteration 557 - count 27850000
iteration 558 - count 27900000
iteration 559 - count 27950000
iteration 560 - count 28000000
iteration 561 - count 28050000
iteration 562 - count 28100000
iteration 563 - count 28150000
iteration 564 - count 28200000
iteration 565 - count 28250000
iteration 566 - count 28300000
iteration 567 - count 28350000
iteration 568 - count 28400000
iteration 569 - count 28450000
iteration 570 - count 28500000
iteration 571 - count 28550000
iteration 572 - count 28600000
iteratio

iteration 806 - count 40300000
iteration 807 - count 40350000
iteration 808 - count 40400000
iteration 809 - count 40450000
iteration 810 - count 40500000
iteration 811 - count 40550000
iteration 812 - count 40600000
iteration 813 - count 40650000
iteration 814 - count 40700000
iteration 815 - count 40750000
iteration 816 - count 40800000
iteration 817 - count 40850000
iteration 818 - count 40900000
iteration 819 - count 40950000
iteration 820 - count 41000000
iteration 821 - count 41050000
iteration 822 - count 41100000
iteration 823 - count 41150000
iteration 824 - count 41200000
iteration 825 - count 41250000
iteration 826 - count 41300000
iteration 827 - count 41350000
iteration 828 - count 41400000
iteration 829 - count 41450000
iteration 830 - count 41500000
iteration 831 - count 41550000
iteration 832 - count 41600000
iteration 833 - count 41650000


In [42]:
submission = pd.read_csv('sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission_nn001.csv', index=False)
submission

,row_id,meter_reading
0,0,222.013596
1,1,99.363014
2,2,10.074864
3,3,345.728882
4,4,1608.276001
5,5,18.422188
6,6,103.637123
7,7,444.614777
8,8,249.868179
9,9,416.182800


In [ ]:
#!kaggle competitions submit -c ashrae-energy-prediction -f submission_nn001.csv -m "Dense NN"

In [ ]:
1+1